## SO1 Remote Trial

Name: Kai Chen

Date: Mar, 2018

## Task1

A comsumer $i$ shops in week $t$ at a retailer $r$. The retailer carries several categories $c$. In each category $c$, the retailer offers multiple products $j$. The comsumers decision process, that is the probability that consumer $i$ purchases $q_{ijt}$ units of product $j$ in week $t$, can be modeled by

$P(Q_{ijt}=q_{ijt}) = P(I_{ict}=1)P(C_{it}=j \mid I_{ict} = 1) P(Q_{ijt} \mid I_{ict}= 1 \bigwedge C_{it} = j)$

### My answers:

#### Question 1
$P(I_{ict}=1) = 0.5$. It is a Bernoulli distribution.

$P(C_{it}=j \mid I_{ict} = 1) = \frac{1}{N_{c}}$, where $N_{c}$ is the number of different products in category $c$. It is a Uniform distribution.

$P(Q_{ijt} \mid I_{ict}= 1 \bigwedge C_{it} = j) = \frac{1}{\sqrt{2\pi\sigma_{ij}^2}} e^{-\frac{(q_{ijt}-\mu_{ij})^2}{2\sigma_{ij}^2}}$, where $\mu_{ij}$ is $\mu_{ij}$ the mean of the number of products $j$ purchased by user $i$ in a week and $\sigma_{ij}$ is the standard deviation of number of product $j$ purchased by user $i$ in a week. This could be a normal distribution.

#### Question 2
In my opinion the advertisement, discount, user revenu, season drive the category purchase incidence.

#### Question 3
Other factors could influence product choice are:
- the purchase history, e.g., if user $i$ purchased product $j$ in the previous weeks, the likehood of choosing product $j$ in this week will be high
- correction between the products, e.g., if user $i$ purchased product $a$, the likehood of choosing product $b$ will be high 
- the changement of product's package, description could also influence the choice.


## Task 2 

The script of my solution can be found in 'task2.py'